# Subgraph Extraction

In this notebook, we will use cuGraph to extract a subgraph from the test graph.  

Notebook Credits
* Original Authors: Bradley Rees
* Created:   10/16/2019
* Last Edit: 12/20/2019

RAPIDS Versions: 0.12.0

Test Hardware
* GV100 32G, CUDA 10.0


## Introduction

Subgraph Extraction allows the extraction of a portion of a graph and returned as a Graph object that follow-on analytics can be applied against.  For example, many of the community detection analytics simply return a list of vertices in a cluster.  In order to run follow-on processing against a cluster, all the edges within that cluster are also needed.  

For this notebook, we will run Louvain to determine the clusters and then extract one of the clusters using Subgraph Extraction.  We will then explore that extracted subgraph.  



To compute the Louvain cluster in cuGraph use: <br>
* __G', df = cugraph.subgraph(G, v)__
    * G: A cugraph.Graph object
    * v: cudf.Series - list of vertex IDs to be extracted

Returns:<br>
* G': A cugraph.Graph object
* df: cudf.DataFrame
    * old - the original vertex ID
    * new - the new renumbered vertex ID



# Notice
Subgraph Extraction is currently not fully functional.  There are two outstanding issues to address the problems

## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [1]:
# Import needed libraries
import cugraph
import cudf

## Read data using cuDF

In [2]:
# Test file    
datafile='../data//karate-data.csv'

In [3]:
# read the data using cuDF
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

#### Run Louvain to find a cluster to extract
Since the focus is on the subgraph extraction portion, we are going to leave the vertex IDs are they are and just accept that there is an extract cluster in the reults with vertex 0

In [4]:
# The louvain algorithm requires that there are vertex weights.  Just use 1.0 
gdf["data"] = 1.0

In [5]:
# create a Graph 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='data')

In [6]:
# Call Louvain on the graph
df, mod = cugraph.louvain(G) 

In [7]:
# How many partitions where found
part_ids = df["partition"].unique()

In [8]:
print(str(len(part_ids)) + " partition detected")

5 partition detected


In [9]:
for p in range(len(part_ids)):
    part = []
    for i in range(len(df)):
        if (df['partition'][i] == p):
            part.append(df['vertex'][i])
    print("Partition " + str(p) + ":")
    print(part)


Partition 0:
[0]
Partition 1:
[1, 2, 3, 4, 8, 10, 12, 13, 14, 18, 20, 22]
Partition 2:
[5, 6, 7, 11, 17]
Partition 3:
[9, 15, 16, 19, 21, 23, 27, 29, 30, 31, 32, 33, 34]
Partition 4:
[24, 25, 26, 28]


### Run Subgraph Extraction
Let's extract cluster 1

In [10]:
vids = df.query("partition == 1")

In [11]:
v = cudf.Series(vids['vertex'])
len(v)

12

In [12]:
subG = cugraph.subgraph(G, v)

In [14]:
print("Main Graph")
print("\tNumber of Vertices: " + str(G.number_of_vertices()))
print("\tNumber of Edges:    " + str(G.number_of_edges()))

Main Graph
	Number of Vertices: 35
	Number of Edges:    156


In [15]:
print("Subgraph")
print("\tNumber of Vertices: " + str(subG.number_of_vertices()))
print("\tNumber of Edges:    " + str(subG.number_of_edges()))

Subgraph
	Number of Vertices: 12
	Number of Edges:    48


In [18]:
coo = cudf.DataFrame()
subDF = subG.view_edge_list()

In [19]:
len(subDF)

48

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___